In [1]:
# pip install simplet5

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

path = "https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv"
df = pd.read_csv(path)
df.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [3]:
df = df.rename(columns={"headlines":"target_text", "text":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
print(df)

                                             source_text  \
0      summarize: Saurav Kant, an alumnus of upGrad a...   
1      summarize: Kunal Shah's credit card bill payme...   
2      summarize: New Zealand defeated India by 8 wic...   
3      summarize: With Aegon Life iTerm Insurance pla...   
4      summarize: Speaking about the sexual harassmen...   
...                                                  ...   
98396  summarize: A CRPF jawan was on Tuesday axed to...   
98397  summarize: 'Uff Yeh', the first song from the ...   
98398  summarize: According to reports, a new version...   
98399  summarize: A new music video shows rapper Snoo...   
98400  summarize: Madhesi Morcha, an alliance of seve...   

                                             target_text  
0      upGrad learner switches to career in ML & Al w...  
1      Delhi techie wins free food from Swiggy for on...  
2      New Zealand end Rohit Sharma-led India's 12-ma...  
3      Aegon life iTerm insurance plan help

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape


((78720, 2), (19681, 2))

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=8,
            max_epochs=3,
            use_gpu=True
           )

/Users/shyam/opt/anaconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Global seed set to 42


In [ ]:
model.load_model("t5","/content/outputs/simplet5-epoch-0-train-loss-1.3018-val-loss-1.0884", use_gpu=True)

text_to_summarize="""summarize: Rahul Gandhi has replied to Goa CM Manohar Parrikar's letter,
which accused the Congress President of using his "visit to an ailing man for political gains".
"He's under immense pressure from the PM after our meeting and needs to demonstrate his loyalty by attacking me,"
Gandhi wrote in his letter. Parrikar had clarified he didn't discuss Rafale deal with Rahul.
"""
model.predict(text_to_summarize)
